In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[47]:


import os
import pandas as pd
import re
import datetime as dt
import PySimpleGUI as sg
from tabulate import tabulate
import os
from pathlib import Path


# In[48]:


def rename_file(p):
    p = Path(p)
    dat_create = []
    for name_path in p.iterdir(): 
        list_component = []
        time_c = os.path.getctime(name_path)
        list_component.append(name_path)
        list_component.append(time_c)
        dat_create.append(list_component)

        dat_create.sort(key=(lambda x: x[1]))

    new_sort_file_name = [i[0] for i in dat_create]
    num = 1
    for name_path in new_sort_file_name:
        name_path.rename(name_path.parent.joinpath(f'{num} '+ name_path.name))
        num += 1
    print('Названия переменованы и форме "название = дата создания файла + название"')


# In[41]:


def chek_num(s):
    s = str(s)
    list_num = re.search(r'\d+', s)
    if list_num:
        return True
    else:
        return False


# In[42]:


def chek_snils(s):
    s = str(s)
    list_num = re.findall(r'\d+', s)
    num = ''.join(list_num)
    if len(num) == 11:
        snils_format = (f"{num[:3]}-{num[3:6]}-{num[6:9]} {num[9:]}")
        return snils_format
    else:
        return None


# In[43]:


def check_last_num(snils):
    snils = str(snils)
    list_snil = re.findall(r'\d+', snils)
    str_snil = ''.join(list_snil)
    num_kontroll = str_snil[-2:]
    num_summer = str_snil[-3::-1]
    
    sum_num = 0
    for index,val in enumerate(num_summer):
        num = index + 1
        sum_num += num * int(val)
    while True:
        if sum_num < 100:
            check_num = str(sum_num)
            break
        elif sum_num == 100 or sum_num == 101:
            check_num = '00'
            break
        elif sum_num > 101:
            check_num = str(sum_num%101)[-2:]
            break
    if len(check_num) == 1:
        check_num = "0"+check_num
    if check_num == num_kontroll:
        return True#, check_num, str_snil[-2:]
    else:
        return False#, check_num, str_snil[-2:]


# In[44]:


def main(directory, path_itog):
    if path_snils != '':
        directory = path_snils
#     elif path_snils = '':
#         path_path_snils
    
    files = os.listdir(directory)
    
    count_csv_xlsx = 0
    for i in files:
        file_format = i.split('.')[-1]
        if file_format == 'csv' or file_format == 'xlsx' or file_format == 'xls':
            count_csv_xlsx += 1
    if count_csv_xlsx == 0:
        print('\n', f'''Не сформирован!!! В папке \n {directory} \n нет файлов csv xlsx \n (выбраный путь для сохранения  {path_itog})''')
        text_elem = window['-text1-']
        text_elem.update(f'''Не сформирован В папке {directory} нет ниодного файла. (выбраный путь для сохранения  {path_itog})''')
        return 0

    
#     if len(files) == 0:
#         text_elem = window['-text1-']
#         text_elem.update(f'''Не сформирован В папке {directory} нет ниодного файла. (выбраный путь для сохранения  {path_itog})''')
#         return 0

        
    print('\n','Просмотренные файлы')
    df = pd.DataFrame() #делаю итоговый
    df_look_fool = pd.DataFrame() # Поиск дурака.
    for i in files:
        name_file = i.split('.') 
        if name_file[-1] == 'xlsx' or name_file[-1] == 'xls':
            df_check = pd.read_excel(f'{directory}/{i}', dtype={1:'str'}, header=None, names=["Номер", "CНИЛС", "None_1", "None_2"], index_col=False)
            df_check['file_name'] = i
            df_look_fool = pd.concat([df_look_fool, df_check], ignore_index=True)

            df_check = pd.DataFrame(df_check.iloc[:, [1, 4]])

            df_check.columns= ['CНИЛС', 'file_name']
            df = pd.concat([df, df_check], ignore_index=True)
        elif name_file[-1] == 'csv':
            df_check = pd.read_csv(f'{directory}/{i}', sep=';', encoding='windows 1251', dtype={1:'str'}, header=None, names=["Номер", "CНИЛС", "None_1", "None_2"], index_col=False)
            df_check['file_name'] = i

            df_look_fool = pd.concat([df_look_fool, df_check], ignore_index=True)
            df_check = pd.DataFrame(df_check.iloc[:, [1, 4]])


            df_check.columns =['CНИЛС', 'file_name']
            df = pd.concat([df, df_check], ignore_index=True)
#         elif name_file[-1] == 'xls':
#             df_check = pd.read_excel(f'{directory}/{i}', dtype={1:'str'}, header=None, names=["Номер", "CНИЛС", "None_1", "None_2"], index_col=False)
#             df_check['file_name'] = i
#             df_look_fool = pd.concat([df_look_fool, df_check], ignore_index=True)

#             df_check = pd.DataFrame(df_check.iloc[:, [1, 4]])

#             df_check.columns= ['CНИЛС', 'file_name']
#             df = pd.concat([df, df_check], ignore_index=True)
        print(i)
    df = df.loc[df['CНИЛС'].apply(chek_num)].dropna(subset=['CНИЛС'], axis=0)
    
    
    df_fool_itog = df_look_fool.loc[(df_look_fool['Номер'].apply(check_last_num) == True) |                  (df_look_fool['None_1'].apply(check_last_num) == True) |                  (df_look_fool['None_2'].apply(check_last_num) == True) 
                 , :]
    
    if df_fool_itog.shape[0] > 0:
        print('\n', 'Возможно ошибка в разположении столбца (не вставлены в итоговый файл)')
        print(tabulate(df_fool_itog, headers='keys', tablefmt='psql'))
        print('Нужно написать им письмо, вставить СНИЛС вручную')
    

        #Поиск ошибок снилс
    df.rename({'CНИЛС':'snils_probably'}, axis=1, inplace=True)
    df['snils'] = df['snils_probably'].apply(chek_snils)
    df['check'] = df.snils.apply(check_last_num)
    
    
    
    df_false = df.query('check==False')
    if df_false.shape[0] > 0:
        print('\n','ОШИБКИ В СНИЛС, не сходится контрольная сумма (не вставлены в итоговый файл)')
        print(tabulate(df_false, headers='keys', tablefmt='psql'))
    else:
        print('\n', 'Все СНЛИС в итоговом файле корректны')

        
    df_good = df[['snils']]
    df_good = df.loc[df.check!=False, ['snils']]
        
    df_good = df_good.dropna(subset=['snils'])                .drop_duplicates(subset='snils')                .reset_index(drop=True)                .reset_index(drop=False)                .rename({'index':'Номер записи (№)', 'snils':'СНИЛС лица, осуществляющего уход'}, axis=1)
    df_good['Номер записи (№)'] = df_good['Номер записи (№)'] + 1
    
    df_good.to_csv(f'{path_itog}/МСП_066_{to_day}.csv', sep=';', encoding='windows 1251', index=False)
    print('\n','Файл сформирован', '\n', 'количество СНИЛС:', len(df_good) ) 
    text_elem = window['-text1-']
    text_elem.update(f'''Файл сформирован из папки \n {directory} \n, в папку \n {path_itog}''')
#     return True
    


# In[52]:


to_day = dt.datetime.today()
to_day = to_day.strftime("%Y-%m-%d")
# to_day = '2021-08-06'

layout = [[sg.Input(size=(30, 2)), sg.FolderBrowse('папка со СНИЛС', key='-folder_snils-')],
          [sg.Input(size=(30, 2)), sg.FolderBrowse('папка сохранения', key='-folder_itog-')],
          [sg.Button('создать файл СНИЛС', enable_events=True, key='-FUNCTION-', font='Helvetica 12')],
#           sg.CB('send email', key='-email-', change_submits=True, default=True)],
          [sg.Text('Сохранение папок с которыми мы будем работать', size=(60, 6), key='-text1-', font='Helvetica 10')],
         [sg.Input(size=(30, 2)), sg.FolderBrowse('Задать стандартную папку со СНИЛС', key='-folder_snils_standart-')],
         [sg.Input(size=(30, 2)), sg.FolderBrowse('Задать стандартную папку сохранения', key='-folder_itog_standart-')],
          [sg.Text('Поменять названия на "название = номер_создания + название"', size=(60, 1), font='Helvetica 10')],
          [sg.Input(size=(15, 1)), sg.FolderBrowse('В папка', key='-path_rename_file-'), sg.Button('Переименовать', key='-rename_file-')],
         [sg.Text('Информация')],
         [sg.Output(size=(100,20))]
         ]
layout1 = [
    [sg.Text('Выберети папку!', size=(30, 1))]
]

layout2 = [
    [sg.Text('Папки с датой сегодня не существует', size=(30, 1))]
]
layout3 = [
    [sg.Text('Файлов в папке не обнаружено', size=(30, 1))]
]



# Create the window
window = sg.Window('Объединение и проверка СНИЛС', layout)
window1 = sg.Window('Ошибка', layout1)
window2 = sg.Window('Ошибка', layout2)
window3 = sg.Window('Ошибка', layout3)

# Event loop
flag_done = False
while True:
    event, values = window.read()
    if event in (sg.WIN_CLOSED, 'Exit'):
        break

    path_snils = values['-folder_snils-']
    path_itog = values['-folder_itog-']
    path_snils_stand = values['-folder_snils_standart-']
    path_itog_stand = values['-folder_itog_standart-']
    path_rename = values['-path_rename_file-']
    
    path_path_snils = ''
    path_true = True

    #Переменование
    if event == '-rename_file-':
        rename_file(path_rename)
        
    
    global directory
    #Стандартный Пути для СЧИТЫВАНИЯ СНИЛС
    if path_snils_stand != '':
        path_path_snils = path_snils_stand
        with open('path_snils_stand.txt', 'w') as f:
            f.write(path_snils_stand)

    #Стандартный путь для СОХРАНЕНИЯ СНИЛС
    if path_itog_stand != '':
        if path_itog == '': 
            path_itog = path_itog_stand
        with open('path_itog_stand.txt', 'w') as f:
            f.write(path_itog_stand)
            
    #Считываю сохраненную папку для сохранения
    try:
        with open('path_itog_stand.txt', 'r') as f:
            path_itog = f.read()
    except:
        print('Нет считывает куда сохранять')
        
    #Создаю ошибку если нет путя для сохранения
    if event == '-FUNCTION-' and path_itog == '':
        event2 = window1.read()
        if event2 in (sg.WIN_CLOSED, 'Exit'):
            break
        
    # Задаю папку откуда брать СНИЛС 
    if path_snils == '':
        try:
            with open('path_snils_stand.txt', 'r') as f:
                path_path_snils = f.read() #здесь задаю папку в которой храняться папки по датам со СНИЛС
        except:
            print('Не считывает откуда скачивать')
        if os.path.isdir(f'{path_path_snils}/{to_day}') is True:
            directory = f'{path_path_snils}/{to_day}'
            path_true = True
        else:
            path_true = False
            directory = False
            if event == '-FUNCTION-':
                window2.read()

#     print(f'{path_path_snils}/{to_day}')
#     print(directory)
            
    #Если путь конкретно задан. То считываю из него. 
    if path_snils != '':
        directory = path_snils
        path_true == True
#     else:
#         directory = 
            #Нужно проверить существует ли папка с датой сегодня.
    # притягиваю папку сохранения поумолчанию 

    print('\n', 'папка для считывания', '\n', directory)
    print('\n', 'папка для сохранения', '\n', path_itog)
    
    # Проверяю задали ли мы папку, есть ли папка в выводе.
    if event == '-FUNCTION-' and path_snils == '' and path_path_snils == '': 
        print('Ошибка 30')
        event2 = window1.read()
        if event2 in (sg.WIN_CLOSED, 'Exit'):
            break
        
    if event == '-FUNCTION-' and path_true == True and directory != False and flag_done == False:
        flag_num = True

        file_save = main(directory, path_itog)
    elif event == '-FUNCTION-':
        event2 = window1.read()
        print(path_true, directory, flag_done)
        print('Ошибка ___')
        if event2 in (sg.WIN_CLOSED, 'Exit'):
            break

# Close the window i.e. release resource
window.close()


# In[ ]:





# In[ ]:






In [1]:
import pandas as pd

In [ ]:
pd.read_excel()